# 영화관 입장권 통합 전산망

일별 박스오피스
https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do

0. https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do 접속
1. #sSearchTo에 2025-09-06 입력 (데이터 추출 시작 기준 일자)
2. 이후 이 날짜를 일주일 전으로 변경 (예를 들어 2025-09-06에서 2025-08-30, 2025-08-23으로 계속 변경)
3. #sSearchFrom에 #sSearchTo의 일주일간의 날짜 입력 (To가 2025-09-06이면 From엔 2025-08-30)
4. 날짜를 변경했으면 #searchForm > div > div.wrap_btn > button 버튼 클릭
5. 로딩되기를 기다림 (이때 time.sleep 함수 대신 webdriver의 기능을 사용)
6. 로딩이 완료되었다면 #content > div.rst_sch > div.tar.marginT7 > a 버튼을 클릭
7. 이루 날짜를 다시 일주일 전으로 변경 -> 계속 반복함
8. 해당 작업 과정을 관찰 할 수 있게 크롬창을 화면에 띄움

In [78]:
import time
import os
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
from glob import glob

In [79]:
# 1. 크롤링 시작 기준 날짜 (YYYY-MM-DD 형식)
START_DATE = "2003-11-08"

# 2. 몇 주간의 데이터를 다운로드할지 설정
NUM_WEEKS = 52*30 # 약 30년치

# 3. KOBIS 일별 박스오피스 URL
URL = "https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do"

In [80]:
def download_boxoffice_data():
    """
    KOBIS 웹사이트에서 설정된 기간 동안의 박스오피스 데이터를 다운로드하는 메인 함수입니다.
    """
    # 웹 드라이버 설정
    DOWNLOAD_FOLDER_PATH = "D:\\공모전\\25-2 비어플\\영화관람객크롤링\\엑셀데이터"  #파일 다운로드 경로 설정
    # [수정] 다운로드 대기 최대 시간(초) 설정
    DOWNLOAD_TIMEOUT = 120

    chrome_options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": DOWNLOAD_FOLDER_PATH}
    chrome_options.add_experimental_option("prefs", prefs)

    # [추가] User-Agent를 GoogleBot으로 설정
    googlebot_user_agent = "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
    chrome_options.add_argument(f"user-agent={googlebot_user_agent}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(URL)
    print(f"페이지 접속 완료: {URL}")
    print(f"파일 다운로드 경로: {DOWNLOAD_FOLDER_PATH}")

    # current_date는 이제 '조회 종료일'을 의미합니다.
    current_date = datetime.strptime(START_DATE, "%Y-%m-%d")

    try:
        for i in range(NUM_WEEKS):
            # 1. [수정] 조회 기간 계산
            # current_date를 조회 종료일(To)로 설정
            end_date = current_date
            # 조회 시작일(From)을 종료일로부터 6일 전으로 설정 (사용자 요청 반영)
            start_date = end_date - timedelta(days=6)

            # 날짜를 문자열로 변환
            end_date_str = end_date.strftime("%Y-%m-%d")
            start_date_str = start_date.strftime("%Y-%m-%d")
            
            print("-" * 30)
            print(f"({i+1}/{NUM_WEEKS}) 작업 시작: {start_date_str} ~ {end_date_str}")

            # 2. [수정] '조회 종료일'(#sSearchTo) 값 입력
            date_to_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#sSearchTo"))
            )
            driver.execute_script("arguments[0].value = arguments[1];", date_to_input, end_date_str)
            print(f"조회 종료일 입력: {end_date_str}")

            # 3. [추가] '조회 시작일'(#sSearchFrom) 값 입력
            date_from_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#sSearchFrom"))
            )
            driver.execute_script("arguments[0].value = arguments[1];", date_from_input, start_date_str)
            print(f"조회 시작일 입력: {start_date_str}")
            
            # 4. '검색' 버튼 클릭
            search_button = driver.find_element(By.CSS_SELECTOR, "#searchForm > div > div.wrap_btn > button")
            search_button.click()
            print("검색 버튼 클릭 완료")

            # --- [수정] Stale Element 오류를 무시하도록 WebDriverWait 설정 ---
            # '엑셀' 버튼을 기다리는 동안 StaleElementReferenceException이 발생하면
            # 오류를 내지 않고 목표 요소가 나타날 때까지 다시 시도합니다.
            wait = WebDriverWait(driver, 10, ignored_exceptions=[StaleElementReferenceException])
            download_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, "//a[text()='엑셀']"))
            )

            # --- [수정] 지능형 다운로드 대기 및 파일 이름 변경 로직 ---
            
            # 1. 다운로드 전, 기존에 남아있을 수 있는 파일을 먼저 정리합니다.
            default_filename = "KOBIS_일별_박스오피스_2025-09-07.xls"
            target_filepath = os.path.join(DOWNLOAD_FOLDER_PATH, default_filename)
            if os.path.exists(target_filepath):
                os.remove(target_filepath)

            # --- 수정 끝 ---
            # 5. '엑셀' 버튼 클릭
            download_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[text()='엑셀']"))
            )
            download_button.click()
            print("엑셀 버튼 클릭!")

            # 6. 다운로드 확인 알림창(Alert) 처리
            WebDriverWait(driver, 10).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            alert.accept()
            print("다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.")

            # 2. 다운로드가 완료될 때까지 지능적으로 대기합니다.
            start_time = time.time()
            while True:
                # .crdownload 파일이 없으면 다운로드가 끝난 것으로 간주합니다.
                if not glob(os.path.join(DOWNLOAD_FOLDER_PATH, '*.crdownload')):
                    # 다운로드된 최종 파일이 존재하는지 한 번 더 확인합니다.
                    if os.path.exists(target_filepath):
                        print("파일 다운로드 완료.")
                        break
                
                # 최대 대기 시간을 초과하면 오류를 발생시키고 종료합니다.
                if time.time() - start_time > DOWNLOAD_TIMEOUT:
                    raise TimeoutError("파일 다운로드 시간이 너무 오래 걸립니다.")
                
                # 0.5초 간격으로 계속 확인합니다.
                time.sleep(0.5)
                
            # --- [추가] 파일 이름 변경 로직 ---
            # time.sleep(5)

            new_filename = f"KOBIS_{start_date_str}_to_{end_date_str}.xls"
            new_filepath = os.path.join(DOWNLOAD_FOLDER_PATH, new_filename)
            os.rename(target_filepath, new_filepath)
            print(f"파일 이름 변경 완료: {new_filename}")

            time.sleep(0.5) #다음 날짜 이동 전 잠시 대기

            # 7. 다음 조회를 위해 날짜를 1주일 전으로 이동
            current_date -= timedelta(days=7)

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    finally:
        print("-" * 30)
        print("모든 작업이 완료되었습니다. 5초 후에 브라우저를 종료합니다.")
        time.sleep(5)
        driver.quit()

In [81]:
download_boxoffice_data()

페이지 접속 완료: https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do
파일 다운로드 경로: D:\공모전\25-2 비어플\영화관람객크롤링\엑셀데이터
------------------------------
(1/1560) 작업 시작: 2003-11-02 ~ 2003-11-08
조회 종료일 입력: 2003-11-08
조회 시작일 입력: 2003-11-02
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-11-02_to_2003-11-08.xls
------------------------------
(2/1560) 작업 시작: 2003-10-26 ~ 2003-11-01
조회 종료일 입력: 2003-11-01
조회 시작일 입력: 2003-10-26
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-10-26_to_2003-11-01.xls
------------------------------
(3/1560) 작업 시작: 2003-10-19 ~ 2003-10-25
조회 종료일 입력: 2003-10-25
조회 시작일 입력: 2003-10-19
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드를 시작합니다.
파일 다운로드 완료.
파일 이름 변경 완료: KOBIS_2003-10-19_to_2003-10-25.xls
------------------------------
(4/1560) 작업 시작: 2003-10-12 ~ 2003-10-18
조회 종료일 입력: 2003-10-18
조회 시작일 입력: 2003-10-12
검색 버튼 클릭 완료
엑셀 버튼 클릭!
다운로드 확인 창 '확인' 클릭 완료. 파일 다운로드